# Taxonomic assignment: Downstream analyses

In [ ]:
pip install openpyxl

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from openpyxl.workbook import Workbook

In [7]:
# load dataset

def load_data(marker, cell, sim):
    raw_data="../raw_data"
    data_path=f"{raw_data}/OTU_summary_tables/otu_summary_table_{marker}_{cell}_{sim}.tsv"
    return pd.read_csv(data_path, sep="\t")

In [8]:
marker="Full18S"
cell="cellCombined"
sim="sim97"

otu_sim97=load_data(marker, cell, sim)

In [9]:
otu_sim97

,ID,A3,Mock,NH1,NH4,Sim17,Sim22,Th16,Th38,Th40,...,Kingdom,Domain,Phyllum,Class,Order,Family,Genus,Species,Percent_similarity,Sequence
0,001220dcf4ee4dc95afac4abde251863,0.0,0.0,0.0,4.0,0.0,0.0,70.0,0.0,13.0,...,Eukaryota,Opisthokonta,Fungi,Mucoromycota,Mucoromycotina,Mucoromycotina_X,NaN,NaN,1.0,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTAAGTATA...
1,008f88c618bceb9d39417cf386f580b0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Archaeplastida,Streptophyta,Zygnemophyceae,Zygnemophyceae_X,NaN,NaN,NaN,0.6,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCCAAGTATA...
2,00b322ec9242b5706a001e8e1bac70d1,0.0,0.0,0.0,0.0,18.0,0.0,0.0,11.0,0.0,...,Eukaryota,Opisthokonta,Fungi,Chytridiomycota,Chytridiomycotina,Chytridiomycetes,NaN,NaN,0.6,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCCAAGTATA...
3,00d075bc876bba64a82a4286b782f9e6,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Alveolata,Perkinsea,Perkinsida,Perkinsida_X,Perkinsida_XX,Perkinsus,Perkinsus_mediterraneus,0.6,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTCAGTTAA...
4,00e806785b6e1012f0ee9a668b2c0a68,0.0,0.0,0.0,9.0,0.0,0.0,36.0,0.0,3.0,...,Eukaryota,Opisthokonta,Metazoa,Tardigrada,Tardigrada_X,Tardigrada_XX,Tardigrada_XXX,Tardigrada_XXX_sp.,0.9,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTCAGTACT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,fdec50692143d169caa7d757b1176e22,0.0,0.0,0.0,0.0,249.0,5917.0,0.0,0.0,2.0,...,Eukaryota,Opisthokonta,Metazoa,Arthropoda,Crustacea,Ostracoda,NaN,NaN,1.0,AGTTATATGCTTGTCTCAAAGATTAAGCCATGCAGGTCTAAGTATA...
556,fed990acd81a8fdc1dc3f97978f3d1b9,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,0.0,...,Eukaryota,Opisthokonta,Metazoa,Nematoda,Enoplea,Enoplea_X,NaN,NaN,1.0,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTAAGCACA...
557,ffa779471a345ae766d4263360e78cda,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Alveolata,Ciliophora,Spirotrichea,Hypotrichia,Hypotrichia_X,Hypotrichia_XX,Hypotrichia_XX_sp.,0.6,AGTCATATGCTTGTCTCAAAGACTAAGCCATGCATGTCTAAGTATA...
558,ffcc862872d5c3f1e3bab70725cc371e,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,...,Eukaryota,Opisthokonta,Fungi,Cryptomycota,Cryptomycotina,Cryptomycotina_X,Cryptomycotina_XX,Cryptomycotina_XX_sp.,0.9,AGTCATATGCTTGTCCCAAAGATTAAGCCATGCATGTCTAAGTATA...


In [10]:
# save as an excel table
otu_sim97.to_excel(f'ASV_summary_table_{marker}_{cell}_{sim}.xlsx')

In [ ]:
asv_data

In [ ]:
asv_data.info()

In [11]:
# vampyrellida ASVs

vamp97 = otu_sim97[otu_sim97['Order'] == 'Vampyrellida']

vamp97[['ID', 'Total', 'A3', 'Mock', 'NH1', 'NH4', 'Sim17', 'Sim22', 'Th16',
   'Th38', 'Th40', 'X17007', 'Order', 'Family', 'Genus', 'Species', 
   'Percent_similarity']]

,ID,Total,A3,Mock,NH1,NH4,Sim17,Sim22,Th16,Th38,Th40,X17007,Order,Family,Genus,Species,Percent_similarity
13,084e07bed6d405882c51c4761b10319c,126,0.0,0.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vampyrellida,Leptophryidae,NaN,NaN,1.0
43,1789528193bfa8745d87c2da6cf3e5d3,205,0.0,205.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vampyrellida,Leptophryidae,NaN,NaN,1.0
44,17dd4ebd91cc236a7ea6c52373384bbe,30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,Vampyrellida,Thalassomyxa_clade,NaN,NaN,0.6
236,70b7cf2cb7c70245a8ea9a860886dbce,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,Vampyrellida,Leptophryidae,NaN,NaN,1.0
272,7ef9d52dc97ffcdb9b27af0b7735d0e0,22,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,Vampyrellida,Leptophryidae,NaN,NaN,1.0
307,91b1f98492eba26f9f791cf7949afb56,56,0.0,0.0,0.0,11.0,5.0,17.0,0.0,18.0,5.0,0.0,Vampyrellida,Leptophryidae,NaN,NaN,1.0
317,93ec799fa70cc9a684fdbec04fb423d8,52,0.0,0.0,0.0,0.0,46.0,6.0,0.0,0.0,0.0,0.0,Vampyrellida,Leptophryidae,NaN,NaN,0.7
346,a1927e68ef1d297f2ebcaa07a4e68de2,274,0.0,274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vampyrellida,Vampyrellidae,Vampyrella,NaN,0.6
372,ae1b764234310efbcee779f359433840,79,0.0,0.0,0.0,0.0,0.0,0.0,79.0,0.0,0.0,0.0,Vampyrellida,NaN,NaN,NaN,1.0
450,d256a3ac10f96874a1d49e7920a2b374,151,0.0,151.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vampyrellida,NaN,NaN,NaN,1.0


In [16]:
vamp97[vamp97['Mock'] != 0]

,ID,A3,Mock,NH1,NH4,Sim17,Sim22,Th16,Th38,Th40,...,Kingdom,Domain,Phyllum,Class,Order,Family,Genus,Species,Percent_similarity,Sequence
43,1789528193bfa8745d87c2da6cf3e5d3,0.0,205.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Rhizaria,Cercozoa,Endomyxa,Vampyrellida,Leptophryidae,NaN,NaN,1.0,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCAAGTCTCTGTATA...
346,a1927e68ef1d297f2ebcaa07a4e68de2,0.0,274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Rhizaria,Cercozoa,Endomyxa,Vampyrellida,Vampyrellidae,Vampyrella,NaN,0.6,AGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTAAGTATA...
450,d256a3ac10f96874a1d49e7920a2b374,0.0,151.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Rhizaria,Cercozoa,Endomyxa,Vampyrellida,NaN,NaN,NaN,1.0,TGTCATATGCTTGTCTAAAAGATTAAGCCATGCAAGTCTAAGTATA...
534,f5e57caab792512d25cfda2451a9fb48,0.0,1759.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Eukaryota,Rhizaria,Cercozoa,Endomyxa,Vampyrellida,Leptophryidae,NaN,NaN,1.0,AGTCATATGCTCGTCTCAAAGATTAAGCCATGCAAGTCTAAGTATA...


In [ ]:
# count plot
%matplotlib inline
sns.set(rc={'figure.figsize':(15,6)})

sns.histplot(data=asv_data, x='Domain');

## Relative Abundance Barplots

In [ ]:
### Import needed modules
#ggplot is used for plotting and is based on the R package ggplot2
#Pandas is a widely-used Python library for statistics, particularly on tabular data. It installed with Anaconda.
#We will use it to read and write tables in .csv format.



In [ ]:
# prepare data

relab_data = asv_data[['ID',
                       'Total', 
                       'A3', 
                       'Mock', 
                       'NH1', 
                       'NH4', 
                       'Sim17', 
                       'Sim22', 
                       'Th16',
                       'Th38', 
                       'Th40', 
                       'X17007',
                      'Domain']]

In [ ]:
relab_data_melt = relab_data.melt(id_vars=['ID', 'Domain'], var_name='Sample', value_name='Abundance')

In [ ]:
%matplotlib inline
sns.set(rc={'figure.figsize':(15,6)})

relab_data.plot(kind ='bar', stacked = True)